Inspired/based on https://github.com/rayidghani/magicloops and https://github.com/dssg/MLforPublicPolicy/blob/master/labs/2019/lab6_feature_generation_sol.ipynb

In [1]:
import pipeline_evictions as pipeline
import ml_loop_evictions as loop

import importlib
import datetime
import pandas as pd
import numpy as np

# Read data

In [2]:
importlib.reload(pipeline)

datafile = "data/tracts.csv"

#Read data, parsing year column to date type
data = pd.read_csv(datafile, parse_dates=['year'])

# Basic Cleaning

In [3]:
# Delete rows that do not have eviction-rate. We do not want to impute this value

data.dropna(subset=['eviction-rate'], inplace=True)
data['eviction-rate'].isnull().values.any()
data.reset_index(inplace=True, drop=True)

# Create outcome label

In [4]:
def get_eviction_rate(df, year,geoid):
    
  data_to_return = df.loc[(df['year'] == year) & (df['GEOID'] == geoid)]
  
  if(data_to_return.empty):
    return 0
  
  return data_to_return['eviction-rate'].iloc[0]

In [5]:
importlib.reload(pipeline)

from dateutil.relativedelta import relativedelta

#Obtain eviction-rate cutoff for the top 10%, for each year
cutoff_10_percent={}
for year in range(2000,2017):
    year = pd.Timestamp(year,1,1)
    cutoff_10_percent[year]=data.loc[data['year'] == year]['eviction-rate'].quantile(.9)
    
top_10_eviction_rate_in_any_next_3_years_column = np.zeros(len(data))
print(top_10_eviction_rate_in_any_next_3_years_column.shape)

#Generate this feature now
# top_10_eviction_rate_last_year_column = np.zeros(len(data))

for index, row in data.iterrows():
  
  #Because the outcome will come from eviction-rate in next 3 years and we have data till 2016,
  #features data bust be from 2013 or before
  
  #We will also limit rows so that they all have one year before them (to calculate to_10_previous_year_feature)
  # & row['year']>=pd.Timestamp(2001,1,1)
  if(row['year']<=pd.Timestamp(2013,1,1)):    
    
    found_year_where_eviction_was_in_top_10_percent=0
    
    #Get eviction for the next 3 years
    for i in range(1,4):
      date_in_i_years = row['year'] + relativedelta(years=i)
      eviction_rate_in_i_years = get_eviction_rate(data, date_in_i_years,row['GEOID'])
    
      top_10_eviction_rate_in_i_years = 1 if eviction_rate_in_i_years>= cutoff_10_percent[date_in_i_years] else 0
      

      
      #If we found one year that meets requirement, we are done with looping
      if(top_10_eviction_rate_in_i_years==1):
        found_year_where_eviction_was_in_top_10_percent=1
        break

    top_10_eviction_rate_in_any_next_3_years_column[index]=found_year_where_eviction_was_in_top_10_percent

    
    #Generating feature now
#     date_last_years = row['year'] - relativedelta(years=i)
#     eviction_rate_last_year = get_eviction_rate(data, date_last_years,row['GEOID'])
#     top_10_eviction_rate_last_year = 1 if eviction_rate_last_year>= cutoff_10_percent[date_last_years] else 0

#     top_10_eviction_rate_last_year_column[index] = top_10_eviction_rate_last_year
    
    
data['top_10_percent_in_any_next_3_years'] = top_10_eviction_rate_in_any_next_3_years_column

label ='top_10_percent_in_any_next_3_years'

#Feature
# data['top_10_percent_last_year']=top_10_eviction_rate_last_year_column



(27889,)


In [6]:
#Debugging. In case we want to add 1 column for each future year with top 10%

      # eviction_rate_in_1_year = np.zeros(len(data))
# cutoff_in_1_year = np.zeros(len(data))

      
#       if(top_10_eviction_rate_in_i_years==1):
#         print(row['GEOID'])
#         print(row['year'])
#         print(date_in_i_years)
#         print(eviction_rate_in_i_years)
#         print(cutoff_10_percent[date_in_i_years])
#         print(top_10_eviction_rate_in_i_years)     

#       if(i==1):
#         eviction_rate_in_1_year[index]=eviction_rate_in_i_years
#         cutoff_in_1_year[index]=cutoff_10_percent[date_in_i_years]

# data['eviction_rate_in_1_year']='eviction_rate_in_1_year'
# data['cutoff_in_1_year']='eviction_rate_in_1_year'      
    

# Create temporal train and test sets

In [69]:
importlib.reload(pipeline)

#Create sets of train and test data, based on different split thresholds
#The split thresholds corresponds to the starting date of the testing data

#Splits according to https://docs.google.com/spreadsheets/d/1ipqsgThz7hdXXyyNpTuqa4J1inc088lop7lhFsAQ_r0/edit#gid=0
split_thresholds = [pd.Timestamp(i,1,1) for i in range (2004, 2014)]

#Indicating which is the column to be used for splitting training and test daata
date_column='year'

#Amount of data used for test set
test_window = relativedelta(years=4)

#Gap needed between training and test set
prediction_horizon = relativedelta(years=3)

#Generate train and test sets
train_test_sets= pipeline.create_temp_validation_train_and_testing_sets(
  data,
  date_column,
  label,
  split_thresholds,
  test_window,
  prediction_horizon)

In [70]:
for c in train_test_sets[5]['x_train'].columns:
  print(c)

GEOID
year
name
parent-location
population
poverty-rate
renter-occupied-households
pct-renter-occupied
median-gross-rent
median-household-income
median-property-value
rent-burden
pct-white
pct-af-am
pct-hispanic
pct-am-ind
pct-asian
pct-nh-pi
pct-multiple
pct-other
eviction-filings
evictions
eviction-rate
eviction-filing-rate
low-flag
imputed
subbed


# Imputation

In [71]:
importlib.reload(pipeline)

#Impute data on continuous columns for each training and test set

#--->PENDING
#In the meantime, imputing all float columns with mean
  
float_columns = [column for column in data.columns if data[column].dtype=='float']

#Do not consider GEOID column nor top_10_percent_in_any_next_3_years
float_columns.remove('name')
float_columns.remove('top_10_percent_in_any_next_3_years')


for train_test_set in train_test_sets:
  train_data = train_test_set['x_train']
  test_data = train_test_set['x_test']

  #fill na values with mean
  pipeline.impute_data(train_data, float_columns)
  pipeline.impute_data(test_data, float_columns)


# Create features

In [72]:
import feature_generation as fg

importlib.reload(pipeline)
importlib.reload(fg)

#We will have to generate features independently for each different train/test set
for train_test_set in train_test_sets:

  train_features = fg.create_features(train_test_set['x_train'])
  test_features = fg.create_features(train_test_set['x_test']) 
  
  #Alternative for just working with default features
  #train_features, test_features = pipeline.create_features(train_test_set)
  
  #Replace raw data in train_test_set with features generated
  #Keep geoid
  #train_test_set['x_train'] = train_features
  
  train_test_set['x_train'] = train_test_set['x_train']['GEOID'].to_frame().reset_index(drop=True).join(train_features)
  train_test_set['x_test'] = train_test_set['x_test']['GEOID'].to_frame().reset_index(drop=True).join(test_features)
  

In [73]:
train_test_sets[3]['x_test'].head()

,GEOID,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,...,county_average_pct-renter-occupied_above_10,county_average_pct-renter-occupied_above_25,county_average_pct-renter-occupied_above_50,county_average_pct-renter-occupied_above_75,county_average_eviction-filing-rate_above_1,county_average_eviction-filing-rate_above_5,county_average_eviction-filing-rate_above_10,county_average_eviction-filing-rate_above_25,county_average_eviction-filing-rate_above_50,county_average_eviction-filing-rate_above_75
0,51001090100,4296.03,7.33,372.0,19.38,702.00,38578.00,221900.00,35.90,94.12,...,1,0,0,0,1,0,0,0,0,0
1,51001090200,5835.96,8.94,720.0,25.61,723.97,42071.98,144156.65,22.50,59.54,...,1,0,0,0,1,0,0,0,0,0
2,51001090300,3189.61,18.91,227.0,26.95,681.12,38282.67,89908.87,23.31,79.84,...,1,0,0,0,1,0,0,0,0,0
3,51001090400,7539.45,17.33,786.0,33.20,742.82,30691.06,94610.17,26.64,29.82,...,1,0,0,0,1,0,0,0,0,0
4,51001090500,3026.45,12.56,344.0,23.78,688.77,40511.90,116809.08,28.35,77.13,...,1,0,0,0,1,0,0,0,0,0


In [74]:
train_test_sets[3]['x_test'].iloc[:,1:]

,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,...,county_average_pct-renter-occupied_above_10,county_average_pct-renter-occupied_above_25,county_average_pct-renter-occupied_above_50,county_average_pct-renter-occupied_above_75,county_average_eviction-filing-rate_above_1,county_average_eviction-filing-rate_above_5,county_average_eviction-filing-rate_above_10,county_average_eviction-filing-rate_above_25,county_average_eviction-filing-rate_above_50,county_average_eviction-filing-rate_above_75
0,4296.03,7.33,372.0,19.38,702.00,38578.00,221900.00,35.90,94.12,0.00,...,1,0,0,0,1,0,0,0,0,0
1,5835.96,8.94,720.0,25.61,723.97,42071.98,144156.65,22.50,59.54,38.30,...,1,0,0,0,1,0,0,0,0,0
2,3189.61,18.91,227.0,26.95,681.12,38282.67,89908.87,23.31,79.84,13.05,...,1,0,0,0,1,0,0,0,0,0
3,7539.45,17.33,786.0,33.20,742.82,30691.06,94610.17,26.64,29.82,39.54,...,1,0,0,0,1,0,0,0,0,0
4,3026.45,12.56,344.0,23.78,688.77,40511.90,116809.08,28.35,77.13,14.09,...,1,0,0,0,1,0,0,0,0,0
5,5733.93,10.40,510.0,21.92,588.35,44285.18,175661.03,22.52,69.12,23.97,...,1,0,0,0,1,0,0,0,0,0
6,5114.34,5.66,454.0,23.96,508.53,43218.91,144166.19,23.30,52.14,43.60,...,1,0,0,0,1,0,0,0,0,0
7,3752.96,4.73,415.0,14.44,775.00,42214.00,136300.00,23.20,58.84,37.38,...,1,0,0,0,1,0,0,0,0,0
8,6.97,7.33,0.0,19.38,702.00,38578.00,221900.00,35.90,94.12,0.00,...,1,0,0,0,1,0,0,0,0,0
9,26.29,8.93,2.0,25.61,724.00,42075.00,144200.00,22.50,59.51,38.33,...,1,0,0,0,1,0,0,0,0,0


# Build Clasifiers and parameters generation

In [ ]:
importlib.reload(pipeline)

#We define the specific models we want to run
models_to_run=['Baseline','DT','LR','RF','NB','BA','AB','GB']#,ET,KNN,SVM'

#Get all posible models and their different sets of parameters
models, parameters_grid = pipeline.get_models_and_parameters('small')

# Loop over models and different training/test sets

In [ ]:
importlib.reload(pipeline)
importlib.reload(loop)

import warnings

warnings.filterwarnings('ignore')

results = loop.iterate_over_models_and_training_test_sets(models_to_run, models, parameters_grid, train_test_sets)
results

# Observe best models for each train/test set, for different metrics

In [ ]:
results

In [ ]:
importlib.reload(pipeline)

#Lets obtain the best model for each train/test set, for each metric
metrics_to_display = ['p_at_5','p_at_10', 'auc-roc']

best_models_per_metric = {}

for metric in metrics_to_display:
    #indices of rows that have max value in specific metric for each train/test set
    idx = results.groupby(['test_set_start_date'])[metric].transform(max) == results[metric]

    #save table of best models at the specific metric
    best_models_per_metric[metric] = results[idx]

### Best models for Precision at 5%

In [ ]:
best_models_per_metric['p_at_5'].iloc[:, [0,2,3,4,11,12,13]]

### Best models for Precision at 10%

In [ ]:
best_models_per_metric['p_at_10'].iloc[:, [0,2,3,4,14,15,16]]

### Best models for AUC-ROC

In [ ]:
best_models_per_metric['auc-roc'].iloc[:, [0,2,3,4,26]]

### Plot of all model types performance at different train/test sets, for the different metrics

In [ ]:
importlib.reload(pipeline)

best_models_per_metric = {}

for metric in metrics_to_display:
    #For each model, find the set of parameters that work the best in each train/test set
    best_models = pipeline.get_best_models_of_each_type_for_each_train_test_set(models_to_run,results,'test_set_start_date', metric)
    pipeline.plot_models_in_time(models_to_run, best_models, metric)
    best_models_per_metric[metric]=best_models
   